In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [3]:
DOCS_LINK = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQCN9pL21lGy3XPBhKwMX7jT1_SG-Sb_4ZWZ1I0Ctd-0vNhtmH4gFKaLsV5jhz4vSjYlQ9NR_fXF_b6/pub?output=xlsx'

In [4]:
with pd.ExcelFile(DOCS_LINK) as excel_file:
        nz_df = excel_file.parse('From Briefings', index_col='Date', parse_dates=True, header=1)
        aus_df = excel_file.parse('Australia', index_col='date', parse_dates=True)
        can_df = excel_file.parse('Canada', index_col='date', parse_dates=True)
        uk_df = excel_file.parse('UK', index_col='date', parse_dates=True)
nz_df = nz_df[~nz_df.isna().all(1)]
us_df = pd.read_csv('https://covidtracking.com/api/us/daily.csv', parse_dates=True, index_col=0)


In [5]:
def make_country_tests_df(df, columns, millions, new_names=['confirmed', 'tests']):
    new_df = df[columns].rename(columns=dict(zip(columns,new_names)))
    for n in new_names:
        new_df[n+'/1M']=new_df[n]/millions
    return new_df

In [6]:
MILLIONS = {'NZ':4.974490, 'US':329.544974, 'Aus':25.663280, 'Can': 37.976754, 'UK':66.435600}
FULL_NAMES = {'NZ':'New Zealand', 'Aus': 'Australia', 'Can': 'Canada', 'UK':'United Kingdom', 'US':'USA'}
COLUMN_NAMES={'NZ':['Confirmed', 'Tests'], 'US':['positive', 'posNeg'], 'Can':['positives', 'tests'], 'Aus': ['cases', 'tests'], 'UK': ['cases', 'tests']}
def make_tests_df(dfs_map):
    tests_df = pd.concat({FULL_NAMES.get(c, c):make_country_tests_df(df, COLUMN_NAMES[c], MILLIONS[c]) for c,df in dfs_map.items()}, sort=True, names=['country']).reset_index()
    tests_df['case rate'] = tests_df['confirmed']/tests_df['tests']
    return tests_df
    
    

In [7]:
tests_df = make_tests_df({'NZ':nz_df, 'Aus':aus_df, 'Can':can_df, 'UK':uk_df, 'US':us_df })
tests_df['nice_date']=tests_df.iloc[:,1].dt.strftime('%e %b %Y')

In [27]:
now = pd.Timestamp('now', tz='Pacific/Auckland').strftime('%e %b %Y %H:%M (%z)')

In [30]:
LABELS = {
    'case rate':'confirmed<br>case rate', 
    'tests/1M':'total tests per 1M population', 
    'confirmed/1M':'confirmed cases<br>per 1M population',
    'title': 'Comparison of confirmed Covid-19 cases and total tests for selected countries normalised by population<br><sm>Last update: </sm>'+now,
}

In [29]:
fig=(px.scatter(tests_df, y='confirmed/1M', x='tests/1M',symbol='country', color='case rate', hover_data=['confirmed', 'tests'], hover_name='nice_date', labels=LABELS, title=LABELS['title'])
    .update_traces(line_width=3, mode='markers+lines', hovertemplate='<b>%{fullData.name}</b> - %{hovertext}<br>confirmed cases=%{customdata[0]} (%{y:.1f} / 1M)<br>total tests=%{customdata[1]} (%{x:.1f} / 1M)<br>confirmed rate=%{marker.color:.1%}')
    .update_layout(coloraxis=dict(colorbar=dict(tickformat='.1%'), colorscale=px.colors.sequential.Sunsetdark_r), legend=dict(x=.025, y=0.95)))
fig

In [10]:
fig.write_html('../Coronavirus_testing_comparison.html', include_plotlyjs='cdn')